# 함수형 API

In [1]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

In [2]:
# Sequential 모델
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

In [3]:
# 함수형 API로 만든 모델
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

In [4]:
model = Model(input_tensor, output_tensor)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [5]:
# 관련되지 않은 입력과 출력으로 모델을 만드는 경우
unrelated_input = Input(shape=(32,))
bad_model = Model(unrelated_input, output_tensor)

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_1:0", shape=(None, 64), dtype=float32) at layer "dense_3". The following previous layers were accessed without issue: []

In [6]:
# 모델 컴파일은 sequential 클래스와 같음
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [7]:
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)
score = model.evaluate(x_train, y_train)

Epoch 1/10
8/8 [==============================] - 0s 2ms/step - loss: 12.1565
Epoch 2/10
8/8 [==============================] - 0s 2ms/step - loss: 12.6723
Epoch 3/10
8/8 [==============================] - 0s 2ms/step - loss: 14.0515
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 16.1460
Epoch 5/10
8/8 [==============================] - 0s 2ms/step - loss: 18.6173
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 20.9853
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 23.8521
Epoch 8/10
8/8 [==============================] - 0s 2ms/step - loss: 27.4395
Epoch 9/10
8/8 [==============================] - 0s 2ms/step - loss: 31.3049
Epoch 10/10
32/32 [==============================] - 0s 2ms/step - loss: 37.8454


In [8]:
score

37.845401763916016

## 다중 입력 모델

In [9]:
# 2개의 입력을 가진 질문-응답 모델의 함수형 API 구현하기
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1) # 인코딩된 질문과 텍스트를 연결

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer) # 모델 객체를 만들고 2개의 입력과 출력을 주입
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [10]:
# 다중 입력 모델에 데이터 주입하기
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length)) # 랜덤한 넘파이 데이터 생성
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)
answers = to_categorical(answers) # 답은 정수가 아닌 원-핫 인코딩된 벡터

# model.fit([text, question], answers, epochs=10, batch_size=128) # 리스트 입력을 사용하여 학습
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128) # 딕셔너리 입력을 사용하여 학습(입력 이름을 지정했을 때만 사용 가능)

Epoch 1/10
8/8 [==============================] - 0s 15ms/step - loss: 6.2146 - acc: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 0s 13ms/step - loss: 6.1964 - acc: 0.0450
Epoch 3/10
8/8 [==============================] - 0s 12ms/step - loss: 6.1429 - acc: 0.0030
Epoch 4/10
8/8 [==============================] - 0s 12ms/step - loss: 6.0693 - acc: 0.0050
Epoch 5/10
8/8 [==============================] - 0s 11ms/step - loss: 5.9997 - acc: 0.0060
Epoch 6/10
8/8 [==============================] - 0s 11ms/step - loss: 5.8861 - acc: 0.0140
Epoch 7/10
8/8 [==============================] - 0s 11ms/step - loss: 5.7830 - acc: 0.0250
Epoch 8/10
8/8 [==============================] - 0s 11ms/step - loss: 5.6944 - acc: 0.0320
Epoch 9/10
8/8 [==============================] - 0s 11ms/step - loss: 5.6372 - acc: 0.0320
Epoch 10/10
8/8 [==============================] - 0s 11ms/step - loss: 5.5576 - acc: 0.0310


## 다중 출력 모델

In [11]:
# 3개의 출력을 가진 함수형 API 구현하기
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128,5,activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.Conv1D(256,5,activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x) # 출력 층에 이름 지정
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

In [12]:
# 다중 출력 모델의 컴파일 옵션: 다중 손실
#model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])
model.compile(optimizer='rmsprop',
             loss={'age': 'mse',
                   'income': 'categorical_crossentropy',
                   'gender': 'binary_crossentropy'}) # 위와 동일하지만 출력 층에 이름을 지정했을 때만 사용 가능

In [13]:
# 다중 출력 모델의 컴파일 옵션: 손실 가중치
# model.compile(optimizer='rmsprop',
#              loss=['mse','categorical_crossentropy','binary_crossentropy'],
#              loss_weights=[0.25,1.,10.])
model.compile(optimizer='rmsprop',
             loss={'age': 'mse',
                   'income': 'categorical_crossentropy',
                   'gender': 'binary_crossentropy'},
             loss_weights={'age': 0.25,
                          'income': 1.,
                          'gender': 10.}) # 위와 동일하지만 출력 층에 이름을 지정했을 때만 사용 가능

In [14]:
# 다중 출력 모델에 데이터 주입하기
posts = np.random.randint(1, vocabulary_size, size=(num_samples, 1000)) # 랜덤한 넘파이 데이터 생성
age_targets = np.random.randint(1, 50, size=num_samples)
income_targets = np.random.randint(0, num_income_groups, size=num_samples)
income_targets = to_categorical(income_targets)
gender_targets = np.random.randint(0, 2, size=num_samples)

model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)

Epoch 1/10
16/16 [==============================] - 3s 187ms/step - loss: 330.0330 - age_loss: 1271.4426 - income_loss: 4.6912 - gender_loss: 0.7481
Epoch 2/10
16/16 [==============================] - 3s 161ms/step - loss: 52.0532 - age_loss: 169.8098 - income_loss: 2.6245 - gender_loss: 0.6976
Epoch 3/10
16/16 [==============================] - 2s 151ms/step - loss: 43.6380 - age_loss: 132.1466 - income_loss: 2.3312 - gender_loss: 0.8270
Epoch 4/10
16/16 [==============================] - 3s 170ms/step - loss: 48.1849 - age_loss: 152.8030 - income_loss: 2.3436 - gender_loss: 0.7641
Epoch 5/10
16/16 [==============================] - 3s 159ms/step - loss: 53.2160 - age_loss: 171.0441 - income_loss: 2.3630 - gender_loss: 0.8092
Epoch 6/10
16/16 [==============================] - 3s 159ms/step - loss: 47.6928 - age_loss: 150.7153 - income_loss: 2.3398 - gender_loss: 0.7674
Epoch 7/10
16/16 [==============================] - 2s 143ms/step - loss: 42.9060 - age_loss: 130.4691 - income_loss

## 층으로 구성된 비순환 유향 그래프

### 인셉션 모듈(V3)

In [15]:
from keras import layers
# 모든 가지는 동일한 스트라이드(2)를 사용. 출력 크기를 동일하게 만들어 하나로 합치기 위함.
branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)
ouput = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

ValueError: Input 0 of layer conv2d is incompatible with the layer: : expected min_ndim=4, found ndim=2. Full shape received: [None, 128]

이 구현 상에서 4개의 가지가 출력하는 텐서의 크기가 조금씩 다름. 실전에서는 padding='same' 옵션을 주어 출력 크기를 통일시킴.

### 잔차 연결

In [ ]:
# 특성 맵의 크기가 같을 때 원본을 그대로 사용하는 잔차 연결
## x는 4D 텐서라고 가정
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

y = layers.add([y, x])

In [ ]:
# 특성 맵의 크기가 다를 때 선형 변환을 사용하여 잔차 연결
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

y = layers.add([y, residual])

## 층 가중치 공유

In [ ]:
# 케라스의 함수형 API로 공유 층(재사용 층)을 사용하는 모델 구현
lstm = layers.LSTM(32) # LSTM 층 객체
left_input = Input(shape=(None, 128)) # 모델의 왼쪽 가지 구성
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_output) # 모델의 오른쪽 가지 구성. 기존 층 객체를 호출하면 가중치가 재사용됨.

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)
 
model = Model([left_input, right_input], predictions) # 모델 객체를 만들고 훈련하기. 이런 모델을 훈련하면 LSTM 층의 가중치는 양쪽 입력을 바탕으로 업데이트됨.
model.fit([left_data, right_data], targets)

In [17]:
# 케라스에서 샴 비전 모델(공유 합성곱 기반 층)을 구현하는 예
from keras import applications

xception_base = applications.Xception(weights=None, include_top=False)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_features = xception_base(right_input) # 같은 비전 모델을 두 번 호출

merged_features = layers.concatenate([left_features, right_features], axis=-1)